In [90]:
setwd('/fs/cbcb-lab/rob/students/noor/Uncertainity/treeTerminusScripts')
suppressPackageStartupMessages(source("tree_helper_function.R"))
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
load("environment/brain_sim_nodtu/mode=gc_bias/mapDf.RData")
# load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
trueCountsDir <- "/fs/cbcb-lab/rob/students/noor/Uncertainity/brain_sim_nodtu/mode=gc_bias/reads_fastq"

trueCountsDf <- lapply(seq(12), function(i) {
    samp <- ifelse(i<=6,i,i-6)
    rep <- ifelse(i<=6,1,2)
    file <- file.path(trueCountsDir, samp, paste(paste("true_counts", rep,sep="_"),"tsv",sep="."))
    df <- read.delim(file, row.names = 1)
})

comp_mard <- function(x, y, cutoff = 0.01) {
    if(length(x) != length(y))
        stop(paste(length(x), length(y)))
    relDiff <- rep(0, length(x))
    zInds <- which(x <= cutoff & y <= cutoff)
#     print(length(zInds)/length(x))
    nzInds <- setdiff(seq_along(x), zInds)
    relDiff[nzInds] <- abs(x[nzInds] - y[nzInds])/(x[nzInds] + y[nzInds])
    return(sum(relDiff)/length(x))
}

In [3]:
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/seBrainSim.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCons.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeCor.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeMC0.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/mean_cons/treeThr.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_len.RData")
load("environment/brain_sim_nodtu/mode=gc_bias/post_type=gibbs_nrep=100_tf=100/updated/comp_trees/lfcCuts_nolen.RData")

In [178]:
dfCor <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfCor) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfMard <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfMard) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

#### Consensus

In [179]:
consTxps <- treeCons$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(consTxps, trueTxps)
mb <- mergeLeaves(treeCons, seBrainSim[allTxps,])
yCons <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCons <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCons)[["counts"]])

[1] "Missing txps 925"


In [180]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [181]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7
dfCor$Sample <- colnames(seBrainSim)
dfCor$Type <- rep("Consensus", ncol(seBrainSim))
dfMard$Sample <- colnames(seBrainSim)
dfMard$Type <- rep("Consensus", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCons[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfCor[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCons[depth==i,j]))
    dfMard[[n]] <- mat[i,]
}



In [182]:
cons1 <- dCuts$Consensus
cons1I <- cons1[cons1 > length(treeCons$tip)]
cons1I <- cons1I + length(tr$tip)-length(treeCons$tip)

cons2 <- dCutsNL$Consensus
cons2I <- cons2[cons2 > length(treeCons$tip)]
cons2I <- cons2I + length(tr$tip)-length(treeCons$tip)
cons2L <- match(treeCons$tip[cons2[cons2 <= length(treeCons$tip)]], tr$tip)
cons1L <- match(treeCons$tip[cons1[cons1 <= length(treeCons$tip)]], tr$tip)
cons1 <- c(cons1L, cons1I)
cons2 <- c(cons2L, cons2I)
length(cons2)

group <- Descendants(tr, length(tr$tip)+1, "children")

dfCor <- cbind(dfCor, lfc_len=diag(cor(trueCountsAgg[cons1,], yAggCons[cons1,],method="spearman")))
dfCor <- cbind(dfCor, lfc_nlen=diag(cor(trueCountsAgg[cons2,], yAggCons[cons2,],method="spearman")))
dfCor <- cbind(dfCor, group=diag(cor(trueCountsAgg[group,], yAggCons[group,],method="spearman")))

dfMard <- cbind(dfMard, lfc_len=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[cons1,j], yAggCons[cons1,j])))
dfMard <- cbind(dfMard, lfc_nlen=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[cons2,j], yAggCons[cons2,j])))
dfMard <- cbind(dfMard, group=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[group,j], yAggCons[group,j])))
rm(cons1,cons2,group)

[1] 87215

#### Correlation Tree

In [183]:
corTxps <- treeCor$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(corTxps, trueTxps)
mb <- mergeLeaves(treeCor, seBrainSim[allTxps,])
yCor <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggCor <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yCor)[["counts"]])

[1] "Missing txps 925"


In [184]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [185]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")


dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Anti-Correlation", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Anti-Correlation", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggCor[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggCor[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}


In [186]:
ac1 <- dCuts$`Anti-Correlation`
ac1I <- ac1[ac1 > length(treeCor$tip)]
ac1I <- ac1I + length(tr$tip)-length(treeCor$tip)
ac1L <- match(treeCor$tip[ac1[ac1 <= length(treeCor$tip)]], tr$tip)
ac1 <- c(ac1L, ac1I)

ac2 <- dCutsNL$`Anti-Correlation`
ac2I <- ac2[ac2 > length(treeCor$tip)]
ac2I <- ac2I + length(tr$tip)-length(treeCor$tip)
ac2L <- match(treeCor$tip[ac2[ac2 <= length(treeCor$tip)]], tr$tip)
ac2 <- c(ac2L, ac2I)

group <- Descendants(tr, length(tr$tip)+1, "children")

dfT <- cbind(dfT, lfc_len=diag(cor(trueCountsAgg[ac1,], yAggCor[ac1,],method="spearman")))
dfT <- cbind(dfT, lfc_nlen=diag(cor(trueCountsAgg[ac2,], yAggCor[ac2,],method="spearman")))
dfT <- cbind(dfT, group=diag(cor(trueCountsAgg[group,], yAggCor[group,],method="spearman")))

dfT2 <- cbind(dfT2, lfc_len=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[ac1,j], yAggCor[ac1,j])))
dfT2 <- cbind(dfT2, lfc_nlen=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[ac2,j], yAggCor[ac2,j])))
dfT2 <- cbind(dfT2, group=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[group,j], yAggCor[group,j])))

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)
length(group)
rm(ac1, ac2,group)

[1] 61286

In [187]:
dfCor %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))
dfMard %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9448149,0.9913943,0.9977744,0.9994178,0.9997756,0.9998456,0.9999731
Consensus,0.9448149,0.9910081,0.9955883,0.9976570,0.9986886,0.9992657,0.9997477


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.09940096,0.06844116,0.02929967,0.01350066,0.006791058,0.004010136,0.002000134
Consensus,0.09940096,0.06253313,0.05639070,0.04831245,0.040397517,0.032071083,0.017485159


#### Mean Consensus

In [188]:
miTxps <- treeMC$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(miTxps, trueTxps)
mb <- mergeLeaves(treeMC, seBrainSim[allTxps,])
yMI <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI)[["counts"]])

[1] "Missing txps 925"


In [189]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [190]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf", ncol(seBrainSim))

# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI[depth==i,j]))
    dfT2[[n]] <- mat[i,]
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI[depth==i,]))
    dfT3[[n]] <- mat[i,]
}



In [191]:
mi1 <- dCuts$MeanInf
mi1I <- mi1[mi1 > length(treeMC$tip)]
mi1I <- mi1I + length(tr$tip)-length(treeMC$tip)
mi1L <- match(treeMC$tip[mi1[mi1 <= length(treeMC$tip)]], tr$tip)
mi1 <- c(mi1L, mi1I)

mi2 <- dCutsNL$MeanInf
mi2I <- mi2[mi2 > length(treeMC$tip)]
mi2I <- mi2I + length(tr$tip)-length(treeMC$tip)
mi2L <- match(treeMC$tip[mi2[mi2 <= length(treeMC$tip)]], tr$tip)
mi2 <- c(mi2L, mi2I)

group <- Descendants(tr, length(tr$tip)+1, "children")

dfT <- cbind(dfT, lfc_len=diag(cor(trueCountsAgg[mi1,], yAggMI[mi1,],method="spearman")))
dfT <- cbind(dfT, lfc_nlen=diag(cor(trueCountsAgg[mi2,], yAggMI[mi2,],method="spearman")))
dfT <- cbind(dfT, group=diag(cor(trueCountsAgg[group,], yAggMI[group,],method="spearman")))

dfT2 <- cbind(dfT2, lfc_len=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[mi1,j], yAggMI[mi1,j])))
dfT2 <- cbind(dfT2, lfc_nlen=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[mi2,j], yAggMI[mi2,j])))
dfT2 <- cbind(dfT2, group=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[group,j], yAggMI[group,j])))

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)
length(group)
rm(mi1, mi2,group)

[1] 33678

#### MeanInf0

In [192]:
mi0Txps <- treeMC0$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(mi0Txps, trueTxps)
mb <- mergeLeaves(treeMC0, seBrainSim[allTxps,])
yMI0 <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggMI0 <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yMI0)[["counts"]])

[1] "Missing txps 603"


In [193]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [194]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Mean Inf0", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Mean Inf0", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggMI0[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggMI0[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}


In [195]:
mi01 <- dCuts$MeanInf0
mi01I <- mi01[mi01 > length(treeMC0$tip)]
mi01I <- mi01I + length(tr$tip)-length(treeMC0$tip)
mi01L <- match(treeMC0$tip[mi01[mi01 <= length(treeMC0$tip)]], tr$tip)
mi01 <- c(mi01L, mi01I)

mi02 <- dCutsNL$MeanInf0
mi02I <- mi02[mi02 > length(treeMC0$tip)]
mi02I <- mi02I + length(tr$tip)-length(treeMC0$tip)
mi02L <- match(treeMC0$tip[mi02[mi02 <= length(treeMC0$tip)]], tr$tip)
mi02 <- c(mi02L, mi02I)

group <- Descendants(tr, length(tr$tip)+1, "children")

dfT <- cbind(dfT, lfc_len=diag(cor(trueCountsAgg[mi01,], yAggMI0[mi01,],method="spearman")))
dfT <- cbind(dfT, lfc_nlen=diag(cor(trueCountsAgg[mi02,], yAggMI0[mi02,],method="spearman")))
dfT <- cbind(dfT, group=diag(cor(trueCountsAgg[group,], yAggMI0[group,],method="spearman")))

dfT2 <- cbind(dfT2, lfc_len=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[mi01,j], yAggMI0[mi01,j])))
dfT2 <- cbind(dfT2, lfc_nlen=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[mi02,j], yAggMI0[mi02,j])))
dfT2 <- cbind(dfT2, group=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[group,j], yAggMI0[group,j])))

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)
length(group)
rm(mi01, mi02,group)

[1] 19427

#### Thr Terminus

In [196]:
thrTxps <- treeThr$tip.label
trueTxps <- Reduce(union, lapply(trueCountsDf, function(df) rownames(df)))
allTxps <- union(thrTxps, trueTxps)
mb <- mergeLeaves(treeThr, seBrainSim[allTxps,])
yThr <- mb[["ySwish"]]
tr <- mb[["tree"]]
yAggThr <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=assays(yThr)[["counts"]])

[1] "Missing txps 925"


In [197]:
trueCountsDfU <- lapply(trueCountsDf, function(df) {
    mtnames <- setdiff(allTxps, rownames(df))
    rbind(df, data.frame(Counts=rep(0,length(mtnames)), row.names = mtnames))
})
rnames <- rownames(trueCountsDfU[[1]])
trueCountsDfU <- lapply(trueCountsDfU, function(df) {
    df <- df[rnames,]
    df
})
trueCountsDfU <- do.call(cbind,trueCountsDfU)

colnames(trueCountsDfU) <- colnames(seBrainSim)
rownames(trueCountsDfU) <- rnames

trueCountsDfU <- trueCountsDfU[tr$tip,]

trueCountsAgg <- computeAggNodesU(tree=tr, nodeID=seq(length(tr$tip)+tr$Nnode), se_counts=trueCountsDfU)

In [198]:
depth <- node.depth(tr,2)
depth[depth > 6] = 7

dfT <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT2 <- data.frame(matrix(nrow=ncol(seBrainSim),ncol=9))
colnames(dfT2) <- c("Sample", paste("Depth",c(1:7), sep="_"), "Type")

dfT$Sample <- colnames(seBrainSim)
dfT$Type <- rep("Thr", ncol(seBrainSim))
dfT2$Sample <- colnames(seBrainSim)
dfT2$Type <- rep("Thr", ncol(seBrainSim))
# head(dfCor)
mat <- matrix(0, ncol=ncol(seBrainSim), nrow=7)
for(i in seq(7)) {
    mat[i,] = diag(cor(trueCountsAgg[depth==i,], yAggThr[depth==i,], method = "spearman"))
    n <- paste("Depth", i, sep="_")
    dfT[[n]] <- mat[i,]
    mat[i,] = sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[depth==i,j], yAggThr[depth==i,j]))
    dfT2[[n]] <- mat[i,]
}


In [199]:
thr1 <- dCuts$Cons_Thr
thr1I <- thr1[thr1 > length(treeThr$tip)]
thr1I <- thr1I + length(tr$tip)-length(treeThr$tip)
thr1L <- match(treeThr$tip[thr1[thr1 <= length(treeThr$tip)]], tr$tip)
thr1 <- c(thr1L, thr1I)

thr2 <- dCutsNL$Cons_Thr
thr2I <- thr2[thr2 > length(treeThr$tip)]
thr2I <- thr2I + length(tr$tip)-length(treeThr$tip)
thr2L <- match(treeThr$tip[thr2[thr2 <= length(treeThr$tip)]], tr$tip)
thr2 <- c(thr2L, thr2I)

group <- Descendants(tr, length(tr$tip)+1, "children")


dfT <- cbind(dfT, lfc_len=diag(cor(trueCountsAgg[thr1,], yAggThr[thr1,],method="spearman")))
dfT <- cbind(dfT, lfc_nlen=diag(cor(trueCountsAgg[thr2,], yAggThr[thr2,],method="spearman")))
dfT <- cbind(dfT, group=diag(cor(trueCountsAgg[group,], yAggThr[group,],method="spearman")))

dfT2 <- cbind(dfT2, lfc_len=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[thr1,j], yAggThr[thr1,j])))
dfT2 <- cbind(dfT2, lfc_nlen=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[thr2,j], yAggThr[thr2,j])))
dfT2 <- cbind(dfT2, group=sapply(seq_along(colnames(seBrainSim)), function(j) comp_mard(trueCountsAgg[group,j], yAggThr[group,j])))

dfCor <- rbind(dfCor, dfT)
dfMard <- rbind(dfMard, dfT2)
length(group)
rm(thr1, thr2,group)

[1] 115824

In [200]:
dfCor %>% group_by(Type) %>% select(starts_with(c("Depth", "lfc", "group"))) %>% summarise_all(.funs = c(mean="mean"))
dfMard %>% group_by(Type) %>% select(starts_with(c("Depth", "lfc", "group"))) %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean,lfc_len_mean,lfc_nlen_mean,group_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9448149,0.9913943,0.9977744,0.9994178,0.9997756,0.9998456,0.9999731,0.9681152,0.9610979,0.8867660
Consensus,0.9448149,0.9910081,0.9955883,0.9976570,0.9986886,0.9992657,0.9997477,0.9965525,0.9741663,0.9992946
Mean Inf,0.9448149,0.9976634,0.9991590,0.9994891,0.9996490,0.9997309,0.9999152,0.9932190,0.9758522,0.9979927
Mean Inf0,0.9446949,0.9986109,0.9996749,0.9998361,0.9998941,0.9999174,0.9999696,0.9984601,0.9771004,0.9999409
Thr,0.9448149,0.9975392,0.9990993,0.9994557,0.9996700,0.9997474,0.9996629,1.0000000,0.9624155,0.9683735


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean,lfc_len_mean,lfc_nlen_mean,group_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.09940096,0.06844116,0.02929967,0.013500659,0.006791058,0.004010136,0.002000134,2.681367e-02,0.06782156,0.006241369
Consensus,0.09940096,0.06253313,0.05639070,0.048312455,0.040397517,0.032071083,0.017485159,2.671775e-02,0.06089270,0.012597737
Mean Inf,0.09940096,0.03485119,0.02631151,0.022779845,0.019636236,0.017965069,0.010082454,3.195391e-02,0.05452047,0.023760917
Mean Inf0,0.09520859,0.02518964,0.01103742,0.006958612,0.004819988,0.003715719,0.001796846,1.713703e-02,0.05024900,0.001945961
Thr,0.09940096,0.04523547,0.01965747,0.012440287,0.007716122,0.005357956,0.005741056,3.183546e-06,0.07873940,0.071476495


In [114]:
dfCor %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))
dfMard %>% group_by(Type) %>% select(starts_with("Depth")) %>% summarise_all(.funs = c(mean="mean"))


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.9448149,0.9913943,0.9977744,0.9994178,0.9997756,0.9998456,0.9999731
Consensus,0.9448149,0.9910081,0.9955883,0.9976570,0.9986886,0.9992657,0.9997477
Mean Inf,0.9448149,0.9976634,0.9991590,0.9994891,0.9996490,0.9997309,0.9999152
Mean Inf0,0.9446949,0.9986109,0.9996749,0.9998361,0.9998941,0.9999174,0.9999696


Adding missing grouping variables: `Type`


Type,Depth_1_mean,Depth_2_mean,Depth_3_mean,Depth_4_mean,Depth_5_mean,Depth_6_mean,Depth_7_mean
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Anti-Correlation,0.09940096,0.06844116,0.02929967,0.013500659,0.006791058,0.004010136,0.002000134
Consensus,0.99993000,0.99955032,0.99992230,0.999961021,0.999964303,0.999986354,0.999999998
Mean Inf,0.09940096,0.03485119,0.02631151,0.022779845,0.019636236,0.017965069,0.010082454
Mean Inf0,0.09520859,0.02518964,0.01103742,0.006958612,0.004819988,0.003715719,0.001796846
